<a href="https://colab.research.google.com/github/neorocke/DataScience/blob/main/%5BSD3_5%5DGenerateTarotImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade diffusers


In [2]:
from google.colab import auth
auth.authenticate_user()
from huggingface_hub import login
login(token='')


In [ ]:
import os
import torch
from diffusers import DiffusionPipeline

# Stable Diffusion XL 베이스 모델 로드
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
)
base.to("cuda")

# Stable Diffusion XL 리파이너 모델 로드
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

unet/config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

In [ ]:

# 이미지 저장을 위한 폴더 생성
output_folder = "tarot_cards_xl_1"
os.makedirs(output_folder, exist_ok=True)

# 타로 카드 이름과 해당 프롬프트를 매핑하는 딕셔너리
tarot_prompts = {
    # Major Arcana (22장)
    "The Fool": "An optimistic young office worker standing at a city crosswalk under bright morning sunlight, gazing upward. Casually holding a briefcase, symbolizing new beginnings and adventure.",
    "The Magician": "A confident businessman seated at a modern desk with a focused gaze. Hands spread over a glowing laptop and scattered papers, symbolizing skill and mastery.",
    "The High Priestess": "A serene, wise woman reading a book in a quiet office library. Soft light streams through a window, symbolizing intuition and knowledge.",
    "The Empress": "A warm and nurturing team leader gesturing welcome to her team amidst lush plants in a bright meeting room, symbolizing creativity and abundance.",
    "The Emperor": "A stern-faced CEO standing beside a large window overlooking the vast cityscape with hands clasped behind his back, symbolizing power and structure.",
    "The Hierophant": "An experienced mentor giving a presentation to attentive colleagues in a conference room, pointing to a screen, symbolizing guidance and tradition.",
    "The Lovers": "Two office colleagues sharing coffee with warm smiles in a cozy café, symbolizing partnership and unity.",
    "The Chariot": "A determined office worker driving through a bustling city street under evening lights, symbolizing willpower and progress.",
    "Strength": "A calm woman sipping coffee with a serene smile amidst a busy office environment, symbolizing inner strength and resilience.",
    "The Hermit": "An introverted office worker gazing at city lights from a window in contemplation, symbolizing introspection and wisdom.",
    "Wheel of Fortune": "An individual near a brightly lit city carousel with reflections of lights in their eyes, symbolizing fate, luck, and life's cycles.",
    "Justice": "A focused lawyer reviewing documents at a desk adorned with balanced scales and legal books, symbolizing fairness and responsibility.",
    "The Hanged Man": "A thoughtful office worker leaning back and looking out the window; their reflection appears inverted on the glass, symbolizing a new perspective and letting go.",
    "Death": "A somber office worker packing personal items into a box, preparing to leave, symbolizing endings and transformation.",
    "Temperance": "An office worker carefully pouring coffee from one cup to another, balancing them, symbolizing harmony and moderation.",
    "The Devil": "An individual with a shadowed face in a dimly lit urban alley, symbolizing temptation and attachment.",
    "The Tower": "A tall skyscraper struck by lightning under a stormy sky, with illuminated windows symbolizing upheaval and sudden change.",
    "The Star": "A hopeful woman gazing at a shining star from her office window with a gentle smile, symbolizing inspiration and hope.",
    "The Moon": "A person walking thoughtfully through a moonlit city street, with a long shadow trailing, symbolizing intuition and mystery.",
    "The Sun": "A joyful office worker walking through a sunlit park in full bloom, symbolizing success and joy.",
    "Judgement": "A professional receiving applause from colleagues in a bright conference room, symbolizing self-discovery and renewal.",
    "The World": "A group of diverse colleagues gathered around a large world map in a modern office, symbolizing accomplishment and unity.",
    # Suit of Cups (14장)
    "Ace of Cups": "A joyful office worker holding a steaming coffee cup that emits a warm glow, symbolizing new emotional beginnings.",
    "Two of Cups": "Two colleagues clinking coffee cups with friendly smiles, symbolizing friendship and partnership.",
    "Three of Cups": "Colleagues celebrating with raised coffee cups and laughter, symbolizing joy and camaraderie.",
    "Four of Cups": "A tired office worker gazing pensively at a coffee cup, with three unused cups beside them, symbolizing apathy and contemplation.",
    "Five of Cups": "A sad office worker looking at a spilled coffee cup, unaware of two remaining cups behind them, symbolizing loss and regret.",
    "Six of Cups": "An individual smiling softly at an old photograph on their desk, bathed in warm light, symbolizing memories and innocence.",
    "Seven of Cups": "An office worker pondering various coffee options displayed above, symbolizing dreams and possibilities.",
    "Eight of Cups": "A person leaving a coffee cup on the desk as they walk away, with a long shadow cast, symbolizing letting go.",
    "Nine of Cups": "A content office worker reclining comfortably while enjoying a coffee, symbolizing satisfaction and fulfillment.",
    "Ten of Cups": "Happy colleagues enjoying coffee together in a bright lounge, with a rainbow light effect symbolizing harmony and happiness.",
    "Page of Cups": "A curious young office worker holding a notebook and pen with a coffee cup beside them, symbolizing curiosity and imagination.",
    "Knight of Cups": "A confident individual walking forward, offering a coffee cup, symbolizing enthusiasm and proposal.",
    "Queen of Cups": "A warm-hearted woman listening attentively to a colleague with a gentle smile, symbolizing empathy and compassion.",
    "King of Cups": "A calm leader mentoring a junior colleague with a benevolent gaze, symbolizing stability and understanding.",
    # Suit of Pentacles (14장)
    "Ace of Pentacles": "A hand holding a glowing coin engraved with a pentacle in the foreground, with a city skyline in the background, symbolizing new financial opportunities.",
    "Two of Pentacles": "An individual skillfully balancing two documents in a figure-eight pattern, symbolizing balance and adaptability.",
    "Three of Pentacles": "Colleagues collaborating intently on architectural blueprints, symbolizing teamwork and skill.",
    "Four of Pentacles": "A person holding onto a stack of coins or a wallet protectively with a cautious expression, symbolizing security and control.",
    "Five of Pentacles": "An office worker outside a bank building with a worried look as soft snow falls, symbolizing hardship and need.",
    "Six of Pentacles": "A generous individual extending coins to others with an outstretched hand; a subtle scale is visible, symbolizing generosity and kindness.",
    "Seven of Pentacles": "An office worker thoughtfully reviewing a growth chart with a patient expression, symbolizing patience and growth.",
    "Eight of Pentacles": "A focused office worker meticulously crafting a model or design with tools around, symbolizing diligence and craftsmanship.",
    "Nine of Pentacles": "An elegantly dressed individual enjoying a serene moment in a rooftop garden, symbolizing wealth and self-sufficiency.",
    "Ten of Pentacles": "A person gazing fondly at a family photo on the office wall, surrounded by symbols of generations, symbolizing legacy and stability.",
    "Page of Pentacles": "A curious young office worker enthusiastically reading a project proposal with a pentacle symbol on the document, symbolizing opportunity and learning.",
    "Knight of Pentacles": "A diligent office worker holding documents carefully with a determined expression, symbolizing commitment and responsibility.",
    "Queen of Pentacles": "A nurturing leader arranging a cozy workspace with personal touches, symbolizing care and stability.",
    "King of Pentacles": "A successful CEO reviewing finances with a satisfied smile, set against an opulent office backdrop, symbolizing achievement and prosperity.",
    # Suit of Swords (14장)
    "Ace of Swords": "An office worker holding up a pen that reflects light, symbolizing clarity and new ideas.",
    "Two of Swords": "An individual seated with a blindfold, holding two crossed pens, symbolizing decision and balance.",
    "Three of Swords": "An office worker with a pained expression looking at a heart-shaped object pierced by three pens on the desk, symbolizing heartbreak and sorrow.",
    "Four of Swords": "A person resting peacefully with arms folded on the desk and head bowed, symbolizing recuperation and rest.",
    "Five of Swords": "A victorious office worker collecting pens with a sly smile as others appear defeated, symbolizing conflict and victory.",
    "Six of Swords": "An individual leaving the office carrying a box towards a brighter area, symbolizing transition and journey.",
    "Seven of Swords": "An office worker stealthily taking documents from a cabinet, glancing cautiously over their shoulder, symbolizing strategy and cunning.",
    "Eight of Swords": "An office worker blindfolded and surrounded by stacks of files, symbolizing restriction and challenge.",
    "Nine of Swords": "An office worker sitting up in bed holding their head with a worried expression; city lights visible through the window, symbolizing anxiety and fear.",
    "Ten of Swords": "A defeated office worker collapsed over a desk with ten pens scattered around, symbolizing ending and release.",
    "Page of Swords": "A young office worker eagerly reading a document with hair flowing in the wind, symbolizing curiosity and intellect.",
    "Knight of Swords": "A determined office worker charging forward with a briefcase and intense expression, symbolizing action and drive.",
    "Queen of Swords": "A sharp-minded woman offering advice to a colleague with a keen gaze, symbolizing intellect and judgment.",
    "King of Swords": "A wise leader standing confidently while holding a pen like a scepter, symbolizing authority and truth.",
    # Suit of Wands (14장)
    "Ace of Wands": "A worker holding a glowing stylus pen, symbolizing inspiration and new ventures.",
    "Two of Wands": "A person holding a digital tablet and looking out of a city window, symbolizing planning and decisions.",
    "Three of Wands": "A worker looking over project files and checking a city skyline, symbolizing progress and foresight.",
    "Four of Wands": "Colleagues celebrating together with a decorated office background, symbolizing joy and harmony.",
    "Five of Wands": "Office workers in a lively debate, symbolizing conflict and competition.",
    "Six of Wands": "A worker receiving applause from colleagues, symbolizing victory and recognition.",
    "Seven of Wands": "A determined person standing their ground during a presentation, symbolizing resilience.",
    "Eight of Wands": "Emails and documents flying across a desk, symbolizing speed and quick decisions.",
    "Nine of Wands": "A weary worker leaning on a stack of documents, symbolizing perseverance.",
    "Ten of Wands": "A worker carrying a heavy stack of files, symbolizing burden and responsibility.",
    "Page of Wands": "An enthusiastic worker with a clipboard, ready to start a new project.",
    "Knight of Wands": "A confident worker walking briskly with determination, symbolizing passion and adventure.",
    "Queen of Wands": "A warm, charismatic leader encouraging colleagues, symbolizing creativity and confidence.",
    "King of Wands": "A visionary executive overseeing a project with authority, symbolizing leadership and vision.",
}

# 공통 프롬프트 요소
common_elements = (
    "Vertical tarot card, Golden Art Nouveau style, Gustav Klimt-inspired, fantasy urban office worker theme, "
    "ornate gold borders, decorative patterns, soft muted colors, warm gold accents, consistent lighting, "
    "detailed textures, single office worker in modern city, expression and posture reflect [card meaning], "
    "card name '[Card Name]' at bottom in elegant font"
)

# 이미지 생성 및 저장
for card_name, description in tarot_prompts.items():
    # [card meaning]을 각 카드의 설명으로 대체
    prompt = common_elements.replace("[card meaning]", description)
    prompt = prompt.replace("[Card Name]", card_name)

    # 전체 프롬프트 출력 (디버깅을 위해)
    print(f"Generating image for: {card_name}")
    print(f"Prompt: {prompt}\n")

    # 총 스텝 수과 베이스 및 리파이너에서 사용할 스텝 비율 설정
    n_steps = 40
    high_noise_frac = 0.8  # 베이스에서 80% 진행, 리파이너에서 나머지 20% 진행

    # 타로 카드 비율에 맞춘 해상도 설정 (예: 800x1200 픽셀)
    height = 1200
    width = 800

    # 베이스 모델 실행
    base_output = base(
        prompt=prompt,
        height=height,
        width=width,
        num_inference_steps=n_steps,
        denoising_end=high_noise_frac,
        output_type="latent",
    )
    latents = base_output["images"]

    # 리파이너 모델 실행
    image = refiner(
        prompt=prompt,
        height=height,
        width=width,
        num_inference_steps=n_steps,
        denoising_start=high_noise_frac,
        image=latents,
    ).images[0]

    # 이미지 저장
    filename = os.path.join(output_folder, f"{card_name}.png")
    image.save(filename)
    print(f"{filename} 저장 완료")

In [ ]:
prompt = "A whimsical and creative image depicting a hybrid creature that is a mix of a waffle and a hippopotamus, basking in a river of melted butter amidst a breakfast-themed landscape. It features the distinctive, bulky body shape of a hippo. However, instead of the usual grey skin, the creature’s body resembles a golden-brown, crispy waffle fresh off the griddle. The skin is textured with the familiar grid pattern of a waffle, each square filled with a glistening sheen of syrup. The environment combines the natural habitat of a hippo with elements of a breakfast table setting, a river of warm, melted butter, with oversized utensils or plates peeking out from the lush, pancake-like foliage in the background, a towering pepper mill standing in for a tree.  As the sun rises in this fantastical world, it casts a warm, buttery glow over the scene. The creature, content in its butter river, lets out a yawn. Nearby, a flock of birds take flight"

image = pipe(
    prompt=prompt,
    negative_prompt="",
    num_inference_steps=28,
    guidance_scale=4.5,
    max_sequence_length=512,
).images[0]

# SD 3.5
https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/stable_diffusion_3

In [1]:
import os
import re
import torch
from diffusers import StableDiffusion3Pipeline
from transformers import CLIPTokenizer
from tqdm import tqdm
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 파이프라인 로드
pipe = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3.5-large",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:

# 토크나이저 로드 (프롬프트 길이 검증용)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

# 시드 고정을 위한 설정
FIXED_SEED = 42
generator = torch.Generator(device="cuda").manual_seed(FIXED_SEED)

# 공통 프롬프트 요소
common_elements = (
    "Vertical tarot card, Golden Art Nouveau style, Gustav Klimt-inspired, "
    "fantasy urban office worker theme, ornate gold borders, decorative patterns, "
    "soft muted colors, warm gold accents, consistent lighting, detailed textures, "
    "single office worker in modern city, expression and posture reflect [Card Meaning], "
    "card name '[Card Name]' at bottom in elegant font"
)


In [17]:

# 프롬프트 생성 함수
def create_prompts(card_name, description):
    """
    주어진 카드 이름과 설명을 바탕으로 CLIP용 프롬프트와 T5용 프롬프트를 생성합니다.
    """
    # CLIP용 짧은 프롬프트 (77 토큰 제한 내)
    first_sentence = description.split('. ')[0]
    clip_prompt = f"{card_name}: {first_sentence}."

    # 토큰 수 검증
    clip_tokens = tokenizer.encode(clip_prompt)
    if len(clip_tokens) > 77:
        logging.warning(f"clip_prompt for '{card_name}' exceeds 77 tokens ({len(clip_tokens)} tokens). Truncating.")
        clip_prompt = tokenizer.decode(clip_tokens[:77], skip_special_tokens=True)

    # T5용 긴 프롬프트 (자세한 설명 + 공통 요소)
    meaning_match = re.search(r"symbolizing ([\w\s, and]+)\.", description)
    if meaning_match:
        card_meaning = meaning_match.group(1)
    else:
        card_meaning = "meaning not specified"
        logging.warning(f"'symbolizing' not found in description for card '{card_name}'. Using default meaning.")

    t5_prompt = f"{description}. " + common_elements.replace("[Card Meaning]", card_meaning).replace("[Card Name]", card_name)

    return clip_prompt, t5_prompt

# 이미지 생성 및 저장 함수
def generate_and_save_image(pipe, card_name, clip_prompt, t5_prompt, generator, guidance_scale, save_dir):
    """
    주어진 프롬프트를 사용하여 이미지를 생성하고 저장합니다.
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 안전한 파일 이름 생성
    safe_card_name = re.sub(r'[^\w\s]', '', card_name).replace(' ', '_')
    image_path = os.path.join(save_dir, f"{safe_card_name}.png")

    # 부정 프롬프트 설정 (선택 사항)
    negative_prompt = "low quality, blurry, distorted, overexposed, underexposed, text, watermark"

    try:
        # 프롬프트 출력
        print(f"---\nCard: {card_name}")
        print(f"CLIP Prompt: {clip_prompt}")
        print(f"T5 Prompt: {t5_prompt}")
        print(f"Guidance Scale: {guidance_scale}\n---\n")

        # 이미지 생성
        image = pipe(
            prompt=clip_prompt,
            prompt_3=t5_prompt,
            negative_prompt=negative_prompt,  # 부정 프롬프트 추가
            height=1200,
            width=800,
            num_inference_steps=100,  # 고해상도 및 디테일을 위해 단계 수 증가
            guidance_scale=guidance_scale,
            generator=generator,  # 시드 고정
            max_sequence_length=512,  # 필요한 경우
        ).images[0]

        # 이미지 저장
        image.save(image_path)
        logging.info(f"Image saved for card '{card_name}' at '{image_path}' with guidance_scale={guidance_scale}")
    except Exception as e:
        logging.error(f"Error generating image for card '{card_name}': {e}")


In [18]:

# 타로 카드 이름과 해당 프롬프트를 매핑하는 딕셔너리
tarot_prompts = {
    # Major Arcana (22장)
    "The Fool": "An optimistic young office worker standing at a city crosswalk under bright morning sunlight, gazing upward. Casually holding a briefcase, symbolizing new beginnings and adventure.",
    "The Magician": "A confident businessman seated at a modern desk with a focused gaze. Hands spread over a glowing laptop and scattered papers, symbolizing skill and mastery.",
    "The High Priestess": "A serene, wise woman reading a book in a quiet office library. Soft light streams through a window, symbolizing intuition and knowledge.",
    "The Empress": "A warm and nurturing team leader gesturing welcome to her team amidst lush plants in a bright meeting room, symbolizing creativity and abundance.",
    "The Emperor": "A stern-faced CEO standing beside a large window overlooking the vast cityscape with hands clasped behind his back, symbolizing power and structure.",
    "The Hierophant": "An experienced mentor giving a presentation to attentive colleagues in a conference room, pointing to a screen, symbolizing guidance and tradition.",
    "The Lovers": "Two office colleagues sharing coffee with warm smiles in a cozy café, symbolizing partnership and unity.",
    "The Chariot": "A determined office worker driving through a bustling city street under evening lights, symbolizing willpower and progress.",
    "Strength": "A calm woman sipping coffee with a serene smile amidst a busy office environment, symbolizing inner strength and resilience.",
    "The Hermit": "An introverted office worker gazing at city lights from a window in contemplation, symbolizing introspection and wisdom.",
    "Wheel of Fortune": "An individual near a brightly lit city carousel with reflections of lights in their eyes, symbolizing fate, luck, and life's cycles.",
    "Justice": "A focused lawyer reviewing documents at a desk adorned with balanced scales and legal books, symbolizing fairness and responsibility.",
    "The Hanged Man": "A thoughtful office worker leaning back and looking out the window; their reflection appears inverted on the glass, symbolizing a new perspective and letting go.",
    "Death": "A somber office worker packing personal items into a box, preparing to leave, symbolizing endings and transformation.",
    "Temperance": "An office worker carefully pouring coffee from one cup to another, balancing them, symbolizing harmony and moderation.",
    "The Devil": "An individual with a shadowed face in a dimly lit urban alley, symbolizing temptation and attachment.",
    "The Tower": "A tall skyscraper struck by lightning under a stormy sky, with illuminated windows symbolizing upheaval and sudden change.",
    "The Star": "A hopeful woman gazing at a shining star from her office window with a gentle smile, symbolizing inspiration and hope.",
    "The Moon": "A person walking thoughtfully through a moonlit city street, with a long shadow trailing, symbolizing intuition and mystery.",
    "The Sun": "A joyful office worker walking through a sunlit park in full bloom, symbolizing success and joy.",
    "Judgement": "A professional receiving applause from colleagues in a bright conference room, symbolizing self-discovery and renewal.",
    "The World": "A group of diverse colleagues gathered around a large world map in a modern office, symbolizing accomplishment and unity.",
    # Suit of Cups (14장)
    "Ace of Cups": "A joyful office worker holding a steaming coffee cup that emits a warm glow, symbolizing new emotional beginnings.",
    "Two of Cups": "Two colleagues clinking coffee cups with friendly smiles, symbolizing friendship and partnership.",
    "Three of Cups": "Colleagues celebrating with raised coffee cups and laughter, symbolizing joy and camaraderie.",
    "Four of Cups": "A tired office worker gazing pensively at a coffee cup, with three unused cups beside them, symbolizing apathy and contemplation.",
    "Five of Cups": "A sad office worker looking at a spilled coffee cup, unaware of two remaining cups behind them, symbolizing loss and regret.",
    "Six of Cups": "An individual smiling softly at an old photograph on their desk, bathed in warm light, symbolizing memories and innocence.",
    "Seven of Cups": "An office worker pondering various coffee options displayed above, symbolizing dreams and possibilities.",
    "Eight of Cups": "A person leaving a coffee cup on the desk as they walk away, with a long shadow cast, symbolizing letting go.",
    "Nine of Cups": "A content office worker reclining comfortably while enjoying a coffee, symbolizing satisfaction and fulfillment.",
    "Ten of Cups": "Happy colleagues enjoying coffee together in a bright lounge, with a rainbow light effect symbolizing harmony and happiness.",
    "Page of Cups": "A curious young office worker holding a notebook and pen with a coffee cup beside them, symbolizing curiosity and imagination.",
    "Knight of Cups": "A confident individual walking forward, offering a coffee cup, symbolizing enthusiasm and proposal.",
    "Queen of Cups": "A warm-hearted woman listening attentively to a colleague with a gentle smile, symbolizing empathy and compassion.",
    "King of Cups": "A calm leader mentoring a junior colleague with a benevolent gaze, symbolizing stability and understanding.",
    # Suit of Pentacles (14장)
    "Ace of Pentacles": "A hand holding a glowing coin engraved with a pentacle in the foreground, with a city skyline in the background, symbolizing new financial opportunities.",
    "Two of Pentacles": "An individual skillfully balancing two documents in a figure-eight pattern, symbolizing balance and adaptability.",
    "Three of Pentacles": "Colleagues collaborating intently on architectural blueprints, symbolizing teamwork and skill.",
    "Four of Pentacles": "A person holding onto a stack of coins or a wallet protectively with a cautious expression, symbolizing security and control.",
    "Five of Pentacles": "An office worker outside a bank building with a worried look as soft snow falls, symbolizing hardship and need.",
    "Six of Pentacles": "A generous individual extending coins to others with an outstretched hand; a subtle scale is visible, symbolizing generosity and kindness.",
    "Seven of Pentacles": "An office worker thoughtfully reviewing a growth chart with a patient expression, symbolizing patience and growth.",
    "Eight of Pentacles": "A focused office worker meticulously crafting a model or design with tools around, symbolizing diligence and craftsmanship.",
    "Nine of Pentacles": "An elegantly dressed individual enjoying a serene moment in a rooftop garden, symbolizing wealth and self-sufficiency.",
    "Ten of Pentacles": "A person gazing fondly at a family photo on the office wall, surrounded by symbols of generations, symbolizing legacy and stability.",
    "Page of Pentacles": "A curious young office worker enthusiastically reading a project proposal with a pentacle symbol on the document, symbolizing opportunity and learning.",
    "Knight of Pentacles": "A diligent office worker holding documents carefully with a determined expression, symbolizing commitment and responsibility.",
    "Queen of Pentacles": "A nurturing leader arranging a cozy workspace with personal touches, symbolizing care and stability.",
    "King of Pentacles": "A successful CEO reviewing finances with a satisfied smile, set against an opulent office backdrop, symbolizing achievement and prosperity.",
    # Suit of Swords (14장)
    "Ace of Swords": "An office worker holding up a pen that reflects light, symbolizing clarity and new ideas.",
    "Two of Swords": "An individual seated with a blindfold, holding two crossed pens, symbolizing decision and balance.",
    "Three of Swords": "An office worker with a pained expression looking at a heart-shaped object pierced by three pens on the desk, symbolizing heartbreak and sorrow.",
    "Four of Swords": "A person resting peacefully with arms folded on the desk and head bowed, symbolizing recuperation and rest.",
    "Five of Swords": "A victorious office worker collecting pens with a sly smile as others appear defeated, symbolizing conflict and victory.",
    "Six of Swords": "An individual leaving the office carrying a box towards a brighter area, symbolizing transition and journey.",
    "Seven of Swords": "An office worker stealthily taking documents from a cabinet, glancing cautiously over their shoulder, symbolizing strategy and cunning.",
    "Eight of Swords": "An office worker blindfolded and surrounded by stacks of files, symbolizing restriction and challenge.",
    "Nine of Swords": "An office worker sitting up in bed holding their head with a worried expression; city lights visible through the window, symbolizing anxiety and fear.",
    "Ten of Swords": "A defeated office worker collapsed over a desk with ten pens scattered around, symbolizing ending and release.",
    "Page of Swords": "A young office worker eagerly reading a document with hair flowing in the wind, symbolizing curiosity and intellect.",
    "Knight of Swords": "A determined office worker charging forward with a briefcase and intense expression, symbolizing action and drive.",
    "Queen of Swords": "A sharp-minded woman offering advice to a colleague with a keen gaze, symbolizing intellect and judgment.",
    "King of Swords": "A wise leader standing confidently while holding a pen like a scepter, symbolizing authority and truth.",
    # Suit of Wands (14장)
    "Ace of Wands": "A worker holding a glowing stylus pen, symbolizing inspiration and new ventures.",
    "Two of Wands": "A person holding a digital tablet and looking out of a city window, symbolizing planning and decisions.",
    "Three of Wands": "A worker looking over project files and checking a city skyline, symbolizing progress and foresight.",
    "Four of Wands": "Colleagues celebrating together with a decorated office background, symbolizing joy and harmony.",
    "Five of Wands": "Office workers in a lively debate, symbolizing conflict and competition.",
    "Six of Wands": "A worker receiving applause from colleagues, symbolizing victory and recognition.",
    "Seven of Wands": "A determined person standing their ground during a presentation, symbolizing resilience.",
    "Eight of Wands": "Emails and documents flying across a desk, symbolizing speed and quick decisions.",
    "Nine of Wands": "A weary worker leaning on a stack of documents, symbolizing perseverance.",
    "Ten of Wands": "A worker carrying a heavy stack of files, symbolizing burden and responsibility.",
    "Page of Wands": "An enthusiastic worker with a clipboard, ready to start a new project.",
    "Knight of Wands": "A confident worker walking briskly with determination, symbolizing passion and adventure.",
    "Queen of Wands": "A warm, charismatic leader encouraging colleagues, symbolizing creativity and confidence.",
    "King of Wands": "A visionary executive overseeing a project with authority, symbolizing leadership and vision.",
}


In [19]:

# 메인 루프
for card_name, description in tqdm(tarot_prompts.items(), desc="Generating Tarot Images"):
    try:
        clip_prompt, t5_prompt = create_prompts(card_name, description)

        # guidance_scale 설정 (여기서는 12로 설정, 필요 시 조정)
        guidance_scale = 12.0  # 원하는 값으로 변경 가능 (예: 7.5, 10, 12)

        # 프롬프트 출력
        print(f"---\nCard: {card_name}")
        print(f"CLIP Prompt: {clip_prompt}")
        print(f"T5 Prompt: {t5_prompt}\n---\n")

        generate_and_save_image(pipe, card_name, clip_prompt, t5_prompt, generator, guidance_scale, "tarot_images_100")
    except Exception as e:
        logging.error(f"Failed to generate image for card '{card_name}': {e}")


Generating Tarot Images:   0%|          | 0/78 [00:00<?, ?it/s]

---
Card: The Fool
CLIP Prompt: The Fool: An optimistic young office worker standing at a city crosswalk under bright morning sunlight, gazing upward.
T5 Prompt: An optimistic young office worker standing at a city crosswalk under bright morning sunlight, gazing upward. Casually holding a briefcase, symbolizing new beginnings and adventure.. Vertical tarot card, Golden Art Nouveau style, Gustav Klimt-inspired, fantasy urban office worker theme, ornate gold borders, decorative patterns, soft muted colors, warm gold accents, consistent lighting, detailed textures, single office worker in modern city, expression and posture reflect new beginnings and adventure, card name 'The Fool' at bottom in elegant font
---

---
Card: The Fool
CLIP Prompt: The Fool: An optimistic young office worker standing at a city crosswalk under bright morning sunlight, gazing upward.
T5 Prompt: An optimistic young office worker standing at a city crosswalk under bright morning sunlight, gazing upward. Casually h

  0%|          | 0/100 [00:00<?, ?it/s]

Generating Tarot Images:   1%|▏         | 1/78 [00:45<58:24, 45.52s/it]

---
Card: The Magician
CLIP Prompt: The Magician: A confident businessman seated at a modern desk with a focused gaze.
T5 Prompt: A confident businessman seated at a modern desk with a focused gaze. Hands spread over a glowing laptop and scattered papers, symbolizing skill and mastery.. Vertical tarot card, Golden Art Nouveau style, Gustav Klimt-inspired, fantasy urban office worker theme, ornate gold borders, decorative patterns, soft muted colors, warm gold accents, consistent lighting, detailed textures, single office worker in modern city, expression and posture reflect skill and mastery, card name 'The Magician' at bottom in elegant font
---

---
Card: The Magician
CLIP Prompt: The Magician: A confident businessman seated at a modern desk with a focused gaze.
T5 Prompt: A confident businessman seated at a modern desk with a focused gaze. Hands spread over a glowing laptop and scattered papers, symbolizing skill and mastery.. Vertical tarot card, Golden Art Nouveau style, Gustav Kl

...

  0%|          | 0/100 [00:00<?, ?it/s]

Generating Tarot Images:  99%|█████████▊| 77/78 [58:37<00:45, 45.72s/it]

---
Card: King of Wands
CLIP Prompt: King of Wands: A visionary executive overseeing a project with authority, symbolizing leadership and vision..
T5 Prompt: A visionary executive overseeing a project with authority, symbolizing leadership and vision.. Vertical tarot card, Golden Art Nouveau style, Gustav Klimt-inspired, fantasy urban office worker theme, ornate gold borders, decorative patterns, soft muted colors, warm gold accents, consistent lighting, detailed textures, single office worker in modern city, expression and posture reflect leadership and vision, card name 'King of Wands' at bottom in elegant font
---

---
Card: King of Wands
CLIP Prompt: King of Wands: A visionary executive overseeing a project with authority, symbolizing leadership and vision..
T5 Prompt: A visionary executive overseeing a project with authority, symbolizing leadership and vision.. Vertical tarot card, Golden Art Nouveau style, Gustav Klimt-inspired, fantasy urban office worker theme, ornate gold bord

  0%|          | 0/100 [00:00<?, ?it/s]

Generating Tarot Images: 100%|██████████| 78/78 [59:23<00:00, 45.69s/it]


In [20]:
# prompt: /content/tarot_images_100 폴더를 압축해서 다운로드

from google.colab import files
import tarfile

# 압축할 폴더 경로
folder_path = "/content/tarot_images_100"

# 압축 파일 이름
tar_filename = "tarot_images_100.tar.gz"

# tar 파일 생성
with tarfile.open(tar_filename, "w:gz") as tar:
    tar.add(folder_path, arcname=os.path.basename(folder_path))

# 파일 다운로드
files.download(tar_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>